Solving with Loops and Dict

In [ ]:
import pyodbc as py
import json

conn = py.connect(
    "DRIVER={SQL SERVER};SERVER=Sardors-laptop\SQLSERVER2025;DATABASE=March11;Trusted_Connection=yes")

cursor = conn.cursor()

cursor.execute("Select * from Contacts")

# cursor.fetchall()        # Har bitta rowni chiqarish uchun fetchall dan foydalanamiz.
# cursor.description       # description columnlar haqidagi info larni chiqaradi.

columns = [column[0] for column in cursor.description]   #Bu bilan har bitta column name larni list qilib olamiz
rows = cursor.fetchall()           # Barcha qatorlarni rows ga olamiz

data = []
for i in rows:
    row_dict = dict(zip(columns,i))      # Har bitta row value uchun dictionary yaratamiz, zip() bilan
    properties = []             # properties ni loop ni ichiga qoyib har bir qator uchun properties listini create qilamiz


    for j in columns:
        if j not in ["VID","Email"]:
            properties.append({"property":j.lower(), "value": str(row_dict[j]) if row_dict[j] is not None else "null"})
# "VID","Email" dan tashqari columnlarni value larni properties listiga dict qilib yozib chiqadi

    entry = {}
    if row_dict.get("VID") is not None:
        entry["vid"] = row_dict["VID"]
    if row_dict.get("Email") is not None:
        entry["email"] = row_dict["Email"]


    entry["properties"] = properties
    data.append(entry)
 


with open("contacts.json", "w") as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False) 


cursor.close()
conn.close()

Solving with SQL query

In [21]:
import pyodbc
import json


conn = pyodbc.connect("DRIVER={SQL Server};SERVER=Sardors-laptop\SQLSERVER2025;DATABASE=March11;Trusted_Connection=yes")
cursor = conn.cursor()


query_columns = """
select column_name
from information_schema.columns 
where table_name = 'Contacts' and column_name not in ('VID', 'Email')
"""
cursor.execute(query_columns)
columns = [row.column_name for row in cursor.fetchall()]


union_sql = " UNION ALL ".join(
    [f"select '{col.lower()}' as property, cast(c.{col} as nvarchar(max)) as value" for col in columns]
)


query = f"""
DECLARE @jsonResult NVARCHAR(MAX);
SET @jsonResult = (
    select 
        CASE WHEN c.VID IS NOT NULL THEN c.VID END AS vid,
        CASE WHEN c.VID IS NULL THEN c.Email END AS email,
        (
            select p.property, p.value
            FROM ({union_sql}) p
            FOR JSON PATH
        ) AS properties
    from Contacts c
    for JSON PATH
);
select @jsonResult;
"""


cursor.execute(query)
result = cursor.fetchone()


if result and result[0]:  
    try:
        json_data = json.loads(result[0].strip()) 
    except json.JSONDecodeError as e:
        print("Error occuring" ,e)
        print("The data: -> ", result[0][:1000]) 
        json_data = []
else:
    json_data = []

print(json.dumps(json_data, indent=4, ensure_ascii=False))


cursor.close()
conn.close()


[
    {
        "vid": "259429",
        "properties": [
            {
                "property": "firstname",
                "value": "Harper"
            },
            {
                "property": "lastname",
                "value": "Wolfberg"
            },
            {
                "property": "website",
                "value": "http://hubspot.com"
            },
            {
                "property": "company",
                "value": "HubSpot"
            },
            {
                "property": "phone",
                "value": "555-122-2323"
            },
            {
                "property": "address",
                "value": "25 First Street"
            },
            {
                "property": "city",
                "value": "Cambridge"
            },
            {
                "property": "state",
                "value": "MA"
            },
            {
                "property": "zip",
                "value": "02139"
            }
      

<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
W:\Users\sardo\AppData\Local\Temp\ipykernel_20280\4163101429.py:5: SyntaxWarning: invalid escape sequence '\S'
  conn = pyodbc.connect("DRIVER={SQL Server};SERVER=Sardors-laptop\SQLSERVER2025;DATABASE=March11;Trusted_Connection=yes")
